In [1]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
import copy
import time
import json
import numpy as np
import matplotlib.pyplot as plt 
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pytorch_metric_learning import losses, miners, testers, samplers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from tqdm import tqdm
import random
import wandb

In [2]:
from models.efficientnet import EffNetModel
from models.swin import SwinTransformerModel
from models.efficientnet_v2 import EffNetV2Model

In [3]:
from datasets_handlers.base_dataset import BaseImageDataset, ClassLabelsMapper
from datasets_handlers.dataset_cub import cub200_dataset
from datasets_handlers.in_shop import in_shop_dataset
from datasets_handlers.stanford_online_products import stanford_products_dataset
from datasets_handlers.google_landmark import google_landmark_dataset

In [4]:
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [5]:
#stanford_train = stanford_products_dataset(split='train')
#stanford_test = stanford_products_dataset(split='test')

In [6]:
cub200_train = cub200_dataset(split='train')
cub200_test = cub200_dataset(split='test')

In [7]:
#inshop_train = in_shop_dataset(split='train')
#inshop_gallery = in_shop_dataset(split='gallery')
#inshop_query = in_shop_dataset(split='query')

In [8]:
#glm_train = google_landmark_dataset()

In [9]:
image_size = 224
train_transform = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.ImageCompression(quality_lower=99, quality_upper=100),
        A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=10, border_mode=0, p=0.7),
        A.Resize(image_size, image_size),
        A.CoarseDropout(max_height=int(image_size * 0.4), max_width=int(image_size * 0.4), max_holes=1, p=0.5),
        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
        ToTensorV2()
    ]
)

test_transform = A.Compose([
    A.Resize(image_size, image_size),
    A.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
    ToTensorV2()
])

In [10]:
dataset_train = BaseImageDataset(ClassLabelsMapper(cub200_train), path_prefix='data/', augmentations=train_transform)
dataset_test = BaseImageDataset(cub200_test, path_prefix='data/', augmentations=test_transform)
#dataset_query = BaseImageDataset(inshop_query, path_prefix='data/', augmentations=test_transform)
#dataset_gallery = BaseImageDataset(inshop_gallery, path_prefix='data/', augmentations=test_transform)
#dataset_query = BaseImageDataset(inshop_query, path_prefix='data/', augmentations=test_transform)

In [11]:
parameters = {
    'dataset:': type(dataset_train.dataset.dataset).__name__,
    'n_epochs': 500,
    'batch_size': 16,
    'lr': 1e-5
}

In [12]:
wandb.init(project="my-test-project", entity="ilya_fedorov", config=parameters)

wandb: Currently logged in as: ilya_fedorov (use `wandb login --relogin` to force relogin)


In [14]:
%%time
train_targets = []
for x, target in cub200_train:
    train_targets.append(target)

CPU times: user 1.62 ms, sys: 0 ns, total: 1.62 ms
Wall time: 1.62 ms


In [15]:
sampler = samplers.MPerClassSampler(
    train_targets, m=4, length_before_new_iter=len(dataset_train)
)

In [16]:
dataloader_train = DataLoader(
    dataset_train,
    batch_size=parameters['batch_size'],
    num_workers=8,
    pin_memory=True,
    sampler=sampler
)

In [17]:
#model = EffNetModel('efficientnet-b0').cuda()
#model = SwinTransformerModel('microsoft/swin-base-patch4-window7-224')
timm_model_name = 'vit_base_patch8_224'
model = EffNetV2Model(768, timm_model_name)
model.cuda()
model = model.train()

In [18]:
embedding_size = model(dataset_train[0][0].unsqueeze(0).cuda()).shape[1]
train_n_classes = len(dataset_train.dataset.labels_map)
model_total_params = sum(p.numel() for p in model.parameters())
print(model_total_params)
print(embedding_size)

86595328
1024


In [70]:
import time

In [71]:
model.eval()

t0 = time.time()
with torch.no_grad():
    s = 0
    for img, label in dataloader_train:
        img = img.cuda()
        out = model(img)
        s += 1
        
        if time.time() - t0 > 1:
            t1 = time.time()
            break

model.train()

EffNetV2Model(
  (model): MlpMixer(
    (stem): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (blocks): Sequential(
      (0): ResBlock(
        (norm1): Affine()
        (linear_tokens): Linear(in_features=196, out_features=196, bias=True)
        (drop_path): Identity()
        (norm2): Affine()
        (mlp_channels): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU()
          (drop1): Dropout(p=0.0, inplace=False)
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop2): Dropout(p=0.0, inplace=False)
        )
      )
      (1): ResBlock(
        (norm1): Affine()
        (linear_tokens): Linear(in_features=196, out_features=196, bias=True)
        (drop_path): Identity()
        (norm2): Affine()
        (mlp_channels): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU()
          (dro

In [72]:
s

21

In [73]:
21 * 16 / (t1 - t0)

331.70046445111893

In [35]:
t1 - t0

1.0221006870269775

In [19]:
loss_args = {'margin': 0.02}
loss_func = losses.TripletMarginLoss(**loss_args)
#loss_func = losses.ArcFaceLoss(embedding_size=embedding_size, num_classes=train_n_classes).cuda()

miner = miners.TripletMarginMiner(**loss_args)

In [21]:
dataset_gallery = dataset_test
dataset_query = dataset_test

skip_first_neighbour = dataset_query == dataset_gallery

In [22]:
wandb.config.update({"skip_first_neighbour": skip_first_neighbour, 
                     "loss": type(loss_func).__name__, 
                     "model": type(model).__name__,
                     "timm_model_name": timm_model_name,
                     "embedding_size": embedding_size,
                     "train_n_classes": train_n_classes,
                     "model_total_parameters": model_total_params,
                     **loss_args
})

In [23]:
optimizer = torch.optim.AdamW(model.parameters(), lr=parameters['lr'])

scheduler = torch.optim.lr_scheduler.LinearLR(optimizer,
                                              start_factor=1.0,
                                              end_factor=0.1,
                                              total_iters=parameters['n_epochs'])

In [24]:
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester()
    return tester.get_all_embeddings(dataset, model)

def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    if train_set != test_set:
        test_embeddings, test_labels = get_all_embeddings(test_set, model)
    else:
        test_embeddings, test_labels = train_embeddings, train_labels
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    accuracies = accuracy_calculator.get_accuracy(
        test_embeddings, train_embeddings, test_labels, train_labels, skip_first_neighbour
    )
    return accuracies

In [25]:
accuracy_calculator = AccuracyCalculator(include=("mean_average_precision", "precision_at_1"), k=5)

In [26]:
train_iter_log_freq = 50

# first to see how it works out of box
model.eval()
test_results = test(dataset_gallery, dataset_query, model, accuracy_calculator)
wandb.log({"test/map@1": test_results['precision_at_1']})
wandb.log({"test/map@5": test_results['mean_average_precision']})
model.train()

i = 0
mean_loss = 0
for epoch in range(parameters['n_epochs']):
    print(f'Starting epoch {epoch}')
    wandb.log({"epoch": epoch})
    
    for data, labels in tqdm(dataloader_train):
        data = data.cuda()
        labels = labels.cuda()
        optimizer.zero_grad()
        embeddings = model(data)
        hard_pairs = miner(embeddings, labels)
        loss = loss_func(embeddings, labels, hard_pairs)
        mean_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        if (i + 1) % train_iter_log_freq == 0:
            mean_loss /= train_iter_log_freq
            wandb.log({"train/loss": mean_loss})
            mean_loss = 0
        i += 1
        
    wandb.log({"train/lr": optimizer.param_groups[0]["lr"]})
    scheduler.step()
    if (epoch + 1) % 10 == 0:
        # query!
        model.eval()
        test_results = test(dataset_gallery, dataset_query, model, accuracy_calculator)
        wandb.log({"test/map@1": test_results['precision_at_1']})
        wandb.log({"test/map@5": test_results['mean_average_precision']})
        model.train()

100%|███████████████████████████████████████████| 95/95 [00:30<00:00,  3.11it/s]


Starting epoch 0


100%|█████████████████████████████████████████| 150/150 [01:11<00:00,  2.10it/s]


Starting epoch 1


100%|█████████████████████████████████████████| 150/150 [01:13<00:00,  2.04it/s]


...

Starting epoch 20


100%|█████████████████████████████████████████| 150/150 [01:05<00:00,  2.27it/s]


Starting epoch 21


 13%|█████▎                                    | 19/150 [00:10<01:09,  1.89it/s]


KeyboardInterrupt: 

In [ ]:
test_results = test(dataset_train, dataset_train, model, accuracy_calculator)

In [22]:
test_results

{'mean_average_precision': 0.8537094955024545,
 'precision_at_1': 0.8352840422495005}